In [1]:
import numpy as np
import networkx as nx
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination
from pgmpy.sampling import BayesianModelSampling
from scipy.stats import beta, dirichlet


## True Causal Model From Environment:

In [2]:
model = BayesianModel([('Enfermedad', 'Final'), ('Tratamiento', 'Final'), ('Tratamiento', 'Reaccion'),
                       ('Reaccion', 'Final')])

cpd_e = TabularCPD(variable='Enfermedad', variable_card=2, values=[[0.6, 0.4]])
cpd_tr = TabularCPD(variable='Tratamiento', variable_card=2, values=[[0.5, 0.5]])

cpd_r_tr = TabularCPD(variable='Reaccion', variable_card=2, 
                   values=[[.7,.4], [.3,.6]], #vivir|farmaco vivir|cirugia #morir|farmaco morir|cirugia
                   evidence=['Tratamiento'],
                   evidence_card=[2])

cpd_f_e_tr_r = TabularCPD(variable='Final', variable_card=2, 
                   values=[[.9,0,.6,0,.3,0,.8,0],  [.1,1,.4,1,.7,1,.2,1]], #vivir|vivir,farmaco,a #morir|
                  evidence=['Enfermedad', 'Tratamiento','Reaccion'],
                  evidence_card=[2, 2,2])

model.add_cpds(cpd_e, cpd_tr, cpd_r_tr, cpd_f_e_tr_r)

infer = VariableElimination(model)
sampling = BayesianModelSampling(model)

In [34]:
class true_causal_model:
    def __init__(self,N,infer,sampling,model):
        self.num_variables=N
        self.infer=infer
        self.sampling=sampling
        self.true_model=model

    def action_simulator(self,chosen_action):
        response=dict()
        response['Enfermedad']=np.random.randint(0,2)
        response['Tratamiento']=chosen_action
        response['Reaccion']=self.infer.map_query(['Reaccion'],evidence={'Tratamiento': response['Tratamiento'],'Enfermedad': response['Enfermedad']})['Reaccion']
        response['Final']=self.infer.map_query(['Final'],evidence={'Tratamiento': response['Tratamiento'],'Enfermedad': response['Enfermedad'],'Reaccion':response['Reaccion']})['Final']
        return(response)

In [159]:
class causal_agent:
    def __init__(self, true_model,model_simulator, max_variables, max_valores):
        self.max_variables=max_variables
        self.local_model=true_model ## Este es el directamente creado con PGMPY, no la clase true_causal_model
        self.infer=VariableElimination(self.local_model)
        self.beliefs=dict()
        self.K=max_valores
        self.simulator=model_simulator.action_simulator
    
    def belief_random_formation(self):
        alpha=np.random.rand(self.K).tolist()
        
        lista=dirichlet.rvs(alpha,size=1).tolist()
        self.beliefs['Enfermedad']=lista[0]
        
        lista=dirichlet.rvs(alpha,size=1).tolist()
        self.beliefs['Tratamiento']=lista[0]
        
        lista_1=dirichlet.rvs(alpha,size=1).tolist()
        lista_2=dirichlet.rvs(alpha,size=1).tolist()
        lista_3=np.array([lista_1[0],lista_2[0]]).transpose()
        lista_4=lista_3.tolist()
        self.beliefs['Reaccion|Tratamiento']=lista_4
        
        lista_1=dirichlet.rvs(alpha,size=1).tolist()
        lista_2=dirichlet.rvs(alpha,size=1).tolist()
        lista_3=dirichlet.rvs(alpha,size=1).tolist()
        lista_4=dirichlet.rvs(alpha,size=1).tolist()
        lista_5=dirichlet.rvs(alpha,size=1).tolist()
        lista_6=dirichlet.rvs(alpha,size=1).tolist()
        lista_7=dirichlet.rvs(alpha,size=1).tolist()
        lista_8=dirichlet.rvs(alpha,size=1).tolist()
        lista_9=np.array([lista_1[0],lista_2[0],lista_3[0],lista_4[0],lista_5[0],lista_6[0],lista_7[0],lista_8[0]]).transpose()
        lista_10=lista_9.tolist()
        self.beliefs['Final|Enfermedad, Tratamiento, Reaccion']=lista_10
        print("Creencias iniciales formadas")
        #return(self.beliefs)
    
    def print_beliefs(self):
        return(self.beliefs)
    
    def fix_causal_model(self):
        model1=self.local_model
        model=model1.copy()
        
        cpd_e=TabularCPD(variable='Enfermedad', variable_card=2, values=[self.beliefs['Enfermedad']])
        cpd_tr=TabularCPD(variable='Tratamiento', variable_card=2, values=[self.beliefs['Tratamiento']])
        cpd_r_tr=cpd_r_tr = TabularCPD(variable='Reaccion', variable_card=2, values=self.beliefs['Reaccion|Tratamiento'], evidence=['Tratamiento'],evidence_card=[2])
        cpd_f_e_tr_r= TabularCPD(variable='Final', variable_card=2, values=self.beliefs['Final|Enfermedad, Tratamiento, Reaccion'],evidence=['Enfermedad', 'Tratamiento','Reaccion'],evidence_card=[2, 2,2])
        model.add_cpds(cpd_e, cpd_tr, cpd_r_tr, cpd_f_e_tr_r)
        
        if (model.check_model()):
            self.local_model=model
            self.infer= VariableElimination(model)
            print("Modelo Causal local creado")
            return(model,infer)
        else:
            print("Error en CPT")
        
    #def do_calculus(): 
    
    def observing_external(self):
        chosen_action=1 #Esto tiene que salir realmente de la funcion del do_calculus
        datos=self.simulator(chosen_action)
        print(datos)
        
    #def prob_data_given_beliefs(self,datos, self.beliefs):
    
    #def belief_bayesian_updating(self,datos,prior_beliefs):
    

In [160]:
modelo=true_causal_model(4,infer,sampling,model)

In [161]:
agente=causal_agent(model,modelo,4,2)

In [162]:
agente.belief_random_formation()

Creencias iniciales formadas


In [163]:
agente.fix_causal_model()

Modelo Causal local creado


(<pgmpy.models.BayesianModel.BayesianModel at 0x1163586d8>,
 <pgmpy.inference.ExactInference.VariableElimination at 0x11629dcf8>)

In [169]:
agente.observing_external()

{'Enfermedad': 1, 'Tratamiento': 1, 'Reaccion': 1, 'Final': 1}


['Enfermedad', 'Final', 'Tratamiento', 'Reaccion']


pgmpy.models.BayesianModel.BayesianModel